In [40]:
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error

In [41]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [42]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [43]:
year = 2022
month = 2
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02}.parquet')

In [44]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [45]:
# Q1
y_val = df['duration']
error = mean_squared_error(y_val,y_pred)
print(error)
print(y_pred.std())

60.6281664401994
5.28140357655334


In [56]:
# Q2
import os 
outputfile=  f'outputpredtict_{year}_{month}.parquet'
df_result = pd.DataFrame()
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result['pred'] = y_pred
df_result.to_parquet(
    outputfile,
    engine='pyarrow',
    compression=None,
    index=False
)

os.stat(outputfile).st_size / 1024000

58.5886845703125

In [59]:
# Q3
!jupyter nbconvert --to python starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to python
[NbConvertApp] Writing 1081 bytes to starter.py


In [68]:
# Q4
# sha256:sha256:065e9673e24e0dc5113e2dd2b4ca30c9d8aa2fa90f4c0597241c93b63130d233

In [ ]:
# Q5
# Run CMD python starter.py 2022 03
# 12.76

In [1]:
# Q6
# 12.83